Introduction

In [ ]:
########################################

# The notebook below contains code to batch generate glacier flowlines from GrIMP velocity data in a loop, and prepare sampling points for velocity extraction.

# Note that that we experienced kernel crashes when trying to process too many glaciers at once (in excess of 15). We recommand splitting the input box coordinates CSVs into smaller chunks and run this notebook multiple times with different input files.

# The notebook involves:
# 1. Importing necessary libraries and setting up directories
# 2. Defining functions for generating flowlines
# 3. Main processing loop: 
  # - parameter setup,
  # - generating seed points from input box coordinates,
  # - generating flowlines for each glacier,
  # - saving flowlines as points and lines shapefiles, and CSVs,
  # - visualising flowlines over velocity data as points and lines,
  # - collecting and saving flowline statistics for each glacier individually and for all glaciers processed in a given chunk combined.
# 4. Extraction of flowlines manually picked for further analysis (manual picking perfmormed in QGIS)
# 5. Visualisation of manually picked flowlines over velocity data as lines and buffered areas.
# 6. Cataloguing of sampling points for velocity extraction by glacier, flowline, and elevation buffer (sampling points chosen at random within an elevation buffer in QGIS

# This file uses glacier-flow-tools library from the PISM project (https://github.com/pism/glacier-flow-tools.git), licensed under GNU GPLv3.

########################################

Imports Section

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gp
from shapely import Point
import pylab as plt

from glacier_flow_tools.interpolation import velocity
from glacier_flow_tools.pathlines import (
    compute_pathline,
    series_to_pathline_geopandas_dataframe,
    pathline_to_line_geopandas_dataframe,
)
from glacier_flow_tools.utils import register_colormaps
register_colormaps()

compute_pathline Function

In [5]:
from typing import Callable, Dict, Tuple, Union

import geopandas as gp
import numpy as np
from numpy import ndarray
from numpy.linalg import norm
from shapely.geometry import LineString, Point
from tqdm import tqdm as tqdm_script
from tqdm.notebook import tqdm as tqdm_notebook
from xarray import DataArray

from glacier_flow_tools.gaussian_random_fields import (
    distrib_normal,
    generate_field,
    power_spectrum,
)
from glacier_flow_tools.geom import distances

def compute_pathline(
    point: Union[list, ndarray, Point],
    f: Callable,
    f_args: Tuple,
    start_time: float = 0.0,
    end_time: float = 1000.0,
    hmin: float = 0.0001,
    hmax: float = 10,
    tol: float = 1e-4,
    v_threshold: float = 0.0,
    notebook: bool = False,
    progress: bool = False,
    progress_kwargs: Dict = {"leave": False, "position": 0},
) -> Tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Compute a pathline using Runge-Kutta-Fehlberg integration.

    This function computes a pathline, which is a trajectory traced by a particle in a fluid flow. The pathline is computed by integrating the velocity field using the Runge-Kutta-Fehlberg method. The function is unit-agnostic, requiring the user to ensure consistency of units. For example, if the velocity field is given in m/yr, the `start_time` and `end_time` are assumed to be in years.

    Parameters
    ----------
    point : list, ndarray, or shapely.Point
        Starting point of the pathline.
    f : callable
        Function that computes the derivative of the state at a given point.
    f_args : tuple
        Additional arguments to pass to the function `f`.
    start_time : float, optional
        The start time of integration. Default is 0.0.
    end_time : float, optional
        The end time of integration. Default is 1000.0.
    hmin : float, optional
        The minimum step size for the integration. Default is 0.0001.
    hmax : float, optional
        The maximum step size for the integration. Default is 10.
    tol : float, optional
        The error tolerance for the integration. Default is 1e-4.
    v_threshold : float, optional
        A velocity threshold below which the solver stops. Default is 0.
    notebook : bool, optional
        If True, a progress bar is displayed in a Jupyter notebook. Default is False.
    progress : bool, optional
        If True, a progress bar is displayed. Default is False.
    progress_kwargs : dict, optional
        Additional keyword arguments for the progress bar. Default is {"leave": False, "position": 0}.

    Returns
    -------
    pts : ndarray
        The points along the pathline.
    velocities : ndarray
        The velocity at each point along the pathline.
    pts_error_estimate : ndarray
        Error estimate at each point along the pathline.

    Examples
    ----------
    >>> import numpy as np
    >>> from shapely.geometry import Point
    >>> def velocity_field(point, t):
    >>>     x, y = point
    >>>     return np.array([-y, x])
    >>> point = [1, 0]
    >>> pts, v, _ = compute_pathline(point, velocity_field, (), start_time=0, end_time=2*np.pi)
    """

    a2 = 2.500000000000000e-01  #  1/4
    a3 = 3.750000000000000e-01  #  3/8
    a4 = 9.230769230769231e-01  #  12/13
    a5 = 1.000000000000000e00  #  1
    a6 = 5.000000000000000e-01  #  1/2

    b21 = 2.500000000000000e-01  #  1/4
    b31 = 9.375000000000000e-02  #  3/32
    b32 = 2.812500000000000e-01  #  9/32
    b41 = 8.793809740555303e-01  #  1932/2197
    b42 = -3.277196176604461e00  # -7200/2197
    b43 = 3.320892125625853e00  #  7296/2197
    b51 = 2.032407407407407e00  #  439/216
    b52 = -8.000000000000000e00  # -8
    b53 = 7.173489278752436e00  #  3680/513
    b54 = -2.058966861598441e-01  # -845/4104
    b61 = -2.962962962962963e-01  # -8/27
    b62 = 2.000000000000000e00  #  2
    b63 = -1.381676413255361e00  # -3544/2565
    b64 = 4.529727095516569e-01  #  1859/4104
    b65 = -2.750000000000000e-01  # -11/40

    r1 = 2.777777777777778e-03  #  1/360
    r3 = -2.994152046783626e-02  # -128/4275
    r4 = -2.919989367357789e-02  # -2197/75240
    r5 = 2.000000000000000e-02  #  1/50
    r6 = 3.636363636363636e-02  #  2/55

    c1 = 1.157407407407407e-01  #  25/216
    c3 = 5.489278752436647e-01  #  1408/2565
    c4 = 5.353313840155945e-01  #  2197/4104
    c5 = -2.000000000000000e-01  # -1/5

    if isinstance(point, Point):
        point = np.squeeze(np.array(point.coords.xy).reshape(1, -1))

    x = point
    t = start_time
    h = hmax

    pts = np.empty((0, len(x)), dtype=float)
    velocities = np.empty((0, len(x)), dtype=float)
    time = np.empty(0, dtype=float)
    error_estimate = np.empty(0, dtype=float)

    pts = np.vstack([pts, x])
    vel = f(point, start_time, *f_args)
    v = np.sqrt(vel[0] ** 2 + vel[1] ** 2)
    velocities = np.vstack([velocities, vel])
    time = np.append(time, start_time)
    error_estimate = np.append(error_estimate, 0.0)

    p_bar = tqdm_notebook if notebook else tqdm_script
    with p_bar(desc="Integrating pathline", total=end_time, **progress_kwargs) if progress else nullcontext():
        while (t < end_time) and (v > v_threshold):

            if np.isclose(t + h, end_time, rtol=1e-5):
                h = end_time - t

            k1 = h * f(x, t, *f_args)
            if np.any(np.isnan(k1)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])
            k2 = h * f(x + b21 * k1, t + a2 * h, *f_args)
            if np.any(np.isnan(k2)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])
            k3 = h * f(x + b31 * k1 + b32 * k2, t + a3 * h, *f_args)
            if np.any(np.isnan(k3)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])
            k4 = h * f(x + b41 * k1 + b42 * k2 + b43 * k3, t + a4 * h, *f_args)
            if np.any(np.isnan(k4)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])
            k5 = h * f(x + b51 * k1 + b52 * k2 + b53 * k3 + b54 * k4, t + a5 * h, *f_args)
            if np.any(np.isnan(k5)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])
            k6 = h * f(x + b61 * k1 + b62 * k2 + b63 * k3 + b64 * k4 + b65 * k5, t + a6 * h, *f_args)
            if np.any(np.isnan(k6)):
                return np.array([[np.nan, np.nan]]), np.array([[np.nan, np.nan]]), np.array([t]), np.array([np.nan])

            r = norm(r1 * k1 + r3 * k3 + r4 * k4 + r5 * k5 + r6 * k6) / h
            if r <= tol:

                pts = np.append(pts, [x], axis=0)
                velocities = np.append(velocities, [vel], axis=0)
                time = np.append(time, t)
                error_estimate = np.append(error_estimate, r)

                t = t + h
                x = x + c1 * k1 + c3 * k3 + c4 * k4 + c5 * k5

                vel = f(x, t, *f_args)
                v = np.sqrt(vel[0] ** 2 + vel[1] ** 2)

            s = (tol / r) ** 0.25
            h = np.minimum(h * s, hmax)

            if (h < hmin) and (t < end_time):
                print(
                    f"Error: Could not converge to the required tolerance {tol:e} with minimum stepsize  {hmin:e} at t={t}"
                )
                continue

    return pts, velocities, time, error_estimate

Batch Prossesing of Flowline Generation

In [ ]:
# === BATCH PROCESSING FOR MULTIPLE GLACIERS ===

import os
from tqdm import tqdm
import rasterio
import geopandas as gp
import numpy as np
from shapely.geometry import Point
import pandas as pd
from contextlib import nullcontext

#Parameters for pathline computation
hmin = 0.01
hmax = 1
tol = 0.1 #was 1 before
start_time = 0
end_time = 1000
v_threshold = 100
spacing = 200  # spacing for seed points, was 500 before, matches GrIMP mosaic resolution


# Input CSV
coords_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Input/Box Coordinates/box_sp_1_v3.csv'
df_coords = pd.read_csv(coords_path)

# For collecting stats later
all_glacier_stats = []

# === PROCESS EACH GLACIER ===
for _, row in tqdm(df_coords.iterrows(), total=len(df_coords), desc="Processing glaciers"):
    try:
        feature_id = row.get('feature_ID', 'unknown')
        glacier_id = row.get('glacier_ID', 'unknown')
        glacier_name = row.get('glacier_name', 'unknown')

        sp_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Input/Starting Points'
        os.makedirs(sp_path, exist_ok=True)

        seed_points = []
        seed_metadata = []
        used_box_nums = set()

        # Identify all box numbers dynamically (even if some unused)
        box_nums = set()
        for col in row.index:
            if col.startswith("coord_1_"):
                num = col.split("_")[2]
                box_nums.add(num)

        # For each box, generate seed points
        for num in sorted(box_nums, key=int):
            coord1_col = f"coord_1_{num}"
            coord2_col = f"coord_2_{num}"

            # Skip if any of the coords are NaN
            if pd.isna(row.get(coord1_col)) or pd.isna(row.get(coord2_col)):
                continue

            # Parse coordinates from strings
            x1_str, y1_str = str(row[coord1_col]).split(",")
            x2_str, y2_str = str(row[coord2_col]).split(",")
            x1, y1 = float(x1_str), float(y1_str)
            x2, y2 = float(x2_str), float(y2_str)

            # Order coordinates
            x_start, x_end = sorted([x1, x2])
            y_start, y_end = sorted([y1, y2])

            # Generate ranges
            x_coords = range(int(x_start), int(x_end), spacing)
            y_coords = range(int(y_start), int(y_end), spacing)

            # Create points and metadata
            if x_coords and y_coords:
                used_box_nums.add(num)

            for x in x_coords:
                for y in y_coords:
                    seed_points.append(Point(x, y))
                    seed_metadata.append({
                        'glacier_ID': glacier_id,
                        'feature_ID': feature_id,
                        'glacier_name': glacier_name,
                        'box_number': num,
                        'x': x,
                        'y': y
                    })

        print(f"For Glacier {glacier_name} (ID:{feature_id}) generated {len(seed_points)} seeding points across {len(used_box_nums)} boxes.")

        # Build GeoDataFrame
        glacier_gdf = gp.GeoDataFrame(seed_metadata, geometry=seed_points, crs="EPSG:3413")

        # Save GeoJSON
        filename = f"sp_{feature_id}.geojson"
        filepath = os.path.join(sp_path, filename)
        glacier_gdf.to_file(filepath, driver='GeoJSON')

        # Keep for next steps
        starting_points_df = glacier_gdf

        # Build GeoDataFrame
        glacier_gdf = gp.GeoDataFrame(seed_metadata, geometry=seed_points, crs="EPSG:3413")

        # Save GeoJSON
        filename = f"sp_{feature_id}.geojson"
        filepath = os.path.join(sp_path, filename)
        glacier_gdf.to_file(filepath, driver='GeoJSON')

        # Keep for next steps
        starting_points_df = glacier_gdf
        

        # === GENERATE FLOWLINES FROM SEEDING POINTS ===

        # Load vx
        with rasterio.open("data/kuba-flowlines/GL_vel_mosaic_Annual_01Dec15_30Nov21_vx_v05.0_nwcw_average.tif") as src_vx:
            Vx = src_vx.read(1)
            transform = src_vx.transform
            xres = transform.a
            yres = -transform.e  # usually negative
            width = src_vx.width
            height = src_vx.height

        # Load vy (assumed same grid)
        with rasterio.open("data/kuba-flowlines/GL_vel_mosaic_Annual_01Dec15_30Nov21_vy_v05.0_nwcw_average.tif") as src_vy:
            Vy = src_vy.read(1)

        # Load vv (assumed same grid)
        with rasterio.open("data/kuba-flowlines/GL_vel_mosaic_Annual_01Dec15_30Nov21_vv_v05.0_nwcw_average.tif") as src_vv:
            Vv = src_vv.read(1)

        # Reverse velocity for backward pathlines
        Vx = -Vx
        Vy = -Vy

        # Build coordinate arrays
        x = np.arange(width) * xres + transform.c
        y = np.arange(height) * -yres + transform.f  # careful: flip if needed

        pathlines = []
        metadata = []

        for idx, df in starting_points_df.iterrows():
            pt = df.geometry.coords[0]
            print(f" → Processing seed point {idx+1}/{len(starting_points_df)}: {pt}")
            glacier_id = df.get("glacier_ID", "unknown")
            feature_id = df.get("feature_ID", "unknown")

            try:
                pathline = compute_pathline(
                    [*pt],
                    velocity,
                    f_args=(Vx, Vy, x, y),
                    hmin=hmin,
                    hmax=hmax,
                    tol=tol,
                    start_time=start_time,
                    end_time=end_time,
                    v_threshold=v_threshold,
                    notebook=True,
                    progress=False,
                )
                pathlines.append(pathline)
                metadata.append(df.drop("geometry", errors="ignore"))

            except RuntimeError as e:
                print(f"Pathline failed for point {pt} feature_ID={feature_id}")
                print(f"   → Reason: {e}")


        # === SAVE FLOWLINES AS POINTS ===

        # Parameters
        fl_points_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as Points'
        os.makedirs(fl_points_path, exist_ok=True)

        # Generate result GeoDataFrame
        result = pd.concat(
            [
                series_to_pathline_geopandas_dataframe(
                    s.drop("geometry", errors="ignore"), pathlines[i]
                )
                for i, (k, s) in enumerate(starting_points_df.iterrows())
                if i < len(pathlines)
            ]
        ).reset_index(drop=True)

        # Filter out broken geometries
        result = result[
            (result.geometry.x > -700_000) & (result.geometry.x < 700_000) &
            (result.geometry.y > -3_000_000) & (result.geometry.y < -1_000_000)
        ]

        # Save GeoJSON
        filename = f"fl_{feature_id}_points.geojson"
        filepath = os.path.join(fl_points_path, filename)
        result.to_file(filepath, driver='GeoJSON')


        # === SAVE FLOWLINES AS CSV ===

        # Parameters
        fl_csv_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as CSV'
        os.makedirs(fl_csv_path, exist_ok=True)

        import warnings

        result_wkt = result.copy()

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            result_wkt["geometry"] = result_wkt["geometry"].apply(lambda geom: geom.wkt)

        filename = f"fl_{feature_id}_points_v3.csv"
        filepath = os.path.join(fl_csv_path, filename)
        result_wkt.to_csv(filepath, index=False)


        # === SAVE FLOWLINES AS LINES ===

        # Parameters
        fl_lines_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as Lines'
        os.makedirs(fl_lines_path, exist_ok=True)

        result_lines = pd.concat(
                [
                    pathline_to_line_geopandas_dataframe(pathlines[k][0], attrs={"pathline_id": [k]})
                    for k, _ in starting_points_df.iterrows()
                ]
            ).reset_index(drop=True)

        # Filter out broken geometries
        result_lines = result_lines[
            (result_lines.geometry.bounds.minx > -700_000) & (result_lines.geometry.bounds.maxx < 700_000) &
            (result_lines.geometry.bounds.miny > -3_000_000) & (result_lines.geometry.bounds.maxy < -1_000_000)
        ]

        # Save GeoJSON
        filename = f"fl_{feature_id}_lines_v3.geojson"
        filepath = os.path.join(fl_lines_path, filename)
        result_lines.to_file(filepath, driver='GeoJSON')

        print(f"For Glacier {glacier_name} (ID:{feature_id}) generated {len(result_lines)} flowlines (with {len(result)} points).")
 

        # === GET FLOWLINES STATISTICS ===

        # Aggregate metrics
        summary = result.groupby("pathline_id").agg(
            v_sum=("v", "sum"),
            v_avg=("v", "mean"),
            num_points=("v", "count"),
            stopped_at_time=("time", "max"),
            mean_error=("error", "mean"),
            mean_distance=("distance", "mean")
        ).reset_index()

        # For stopped_at_distance, get last point per pathline
        last_points = result.sort_values("time").groupby("pathline_id").last().reset_index()
        last_points = last_points[["pathline_id", "distance_from_origin"]].rename(
            columns={"distance_from_origin": "stopped_at_distance"}
        )

        # Merge summary and last point info
        summary = pd.merge(summary, last_points, on="pathline_id")

        # Get metadata from the first point
        first_points = result.sort_values("time").groupby("pathline_id").first().reset_index()
        first_points = first_points.rename(columns={
            "x": "x_start",
            "y": "y_start"
        })

        meta_cols = ["pathline_id", "glacier_ID", "feature_ID", "glacier_name", "x_start", "y_start"]
        first_points = first_points[meta_cols]

        # Merge everything
        flowline_stats = pd.merge(first_points, summary, on="pathline_id")

        # Reorder columns as requested
        desired_order = [
            "pathline_id",
            "glacier_ID",
            "feature_ID",
            "glacier_name",
            "x_start",
            "y_start",
            "num_points",
            "v_sum",
            "v_avg",
            "mean_error",
            "mean_distance",
            "stopped_at_distance",
            "stopped_at_time"
        ]
        flowline_stats = flowline_stats[desired_order]

        # Output path
        stats_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Statistics/Flowlines'
        os.makedirs(stats_path, exist_ok=True)
        filename = f"stats_fl_{first_points['feature_ID'].iloc[0]}_v3.csv"
        filepath = os.path.join(stats_path, filename)

        # Save CSV
        flowline_stats.to_csv(filepath, index=False)
        print(f"For Glacier {glacier_name} (ID:{feature_id}) saved flowline statistics.")


        # === SAVE FASTEST FLOWLINES AS LINES ===

        # Identify pathline_ids with highest v_sum and v_mean
        max_sum_id = flowline_stats.sort_values("v_sum", ascending=False).iloc[0]["pathline_id"]
        max_avg_id = flowline_stats.sort_values("v_avg", ascending=False).iloc[0]["pathline_id"]

        # Create output directories if needed
        out_dir_sum = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/Fastest Sum'
        os.makedirs(out_dir_sum, exist_ok=True)
        out_dir_avg = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/Fastest Avg'
        os.makedirs(out_dir_avg, exist_ok=True)

        # Extract the corresponding line geometries
        line_max_sum = pathline_to_line_geopandas_dataframe(
            pathlines[max_sum_id][0], attrs={"pathline_id": [max_sum_id]}
        )
        line_max_avg = pathline_to_line_geopandas_dataframe(
            pathlines[max_avg_id][0], attrs={"pathline_id": [max_avg_id]}
        )

        # Save as GeoJSON
        filename_sum = f"fl_{feature_id}_lines_max_sum_v3.geojson"
        filepath_sum = os.path.join(out_dir_sum, filename_sum)
        line_max_sum.to_file(filepath_sum, driver='GeoJSON')

        filename_avg = f"fl_{feature_id}_lines_max_avg_v3.geojson"
        filepath_avg = os.path.join(out_dir_avg, filename_avg)
        line_max_avg.to_file(filepath_avg, driver='GeoJSON')


        # === SAVE PLOT WITH DOTS ===

        # Create output folder if not exists
        plot_points_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Graphs/with Points'
        os.makedirs(plot_points_path, exist_ok=True)

        # Compute square bounds
        minx, miny, maxx, maxy = result.total_bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2

        width = maxx - minx
        height = maxy - miny
        max_extent = max(width, height) / 2 + 5000  # buffer of 5 km on each side

        minx = center_x - max_extent
        maxx = center_x + max_extent
        miny = center_y - max_extent
        maxy = center_y + max_extent

        # Flip y and Vv if needed (raster is top-down)
        if y[0] > y[-1]:
            y = y[::-1]
            Vv = Vv[::-1, :]

        # Wrap into xarray DataArray
        Vv = xr.DataArray(Vv, coords=[("y", y), ("x", x)])
        Vv_crop = Vv.sel(x=slice(minx, maxx), y=slice(miny, maxy))

        # Compute ratio for correct aspect
        width = maxx - minx
        height = maxy - miny
        # Check for valid numbers
        if width > 0 and height > 0:
            ratio = height / width
        else:
            raise ValueError("Invalid bounds: cannot compute aspect ratio.")

        NWCW = gp.read_file('data/kuba-flowlines/NW-CW_Basins.shp')
        contour100m = gp.read_file('data/kuba-flowlines/contour_100masl_arcticdem_10m_v4.1_nwcw.shp')
        contour1500m = gp.read_file('data/kuba-flowlines/contour_1500masl_arcticdem_10m_v4.1_nwcw.shp')
        contour = gp.read_file('data/kuba-flowlines/contour_every100_arcticdem_10m_v4.1_nwcw.shp')

        fig, ax = plt.subplots(1, figsize=(12, 12))
        ax.set_aspect('equal', adjustable='box')

        # Background raster
        Vv_crop.plot(
            ax=ax,
            cmap="speed_colorblind",
            vmin=10,
            vmax=1500,
            zorder=1,
            cbar_kwargs={
                "shrink": 0.6,
                "label": r"velocity [m a$^{-1}$]",
                "pad": 0.02
            }
        )

        # Contours
        contour.plot(ax=ax, color="b", edgecolor="k", linewidth=0.25, zorder=2)
        contour100m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)
        contour1500m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)

        # NWCW outline (on top of contours)
        NWCW.plot(ax=ax, facecolor="none", edgecolor="g", linewidth=1, zorder=3)

        # Flowlines
        result.plot(ax=ax, color="w", edgecolor="k", linewidth=0.00, markersize = 0.5, zorder=4)

        # Starting points (on top)
        starting_points_df.plot(ax=ax, color="black", edgecolor="k", linewidth=0.01, markersize=0.5, zorder=5)

        # Labels
        ax.set_xlabel("x coordinate of projection [m]", fontsize=12)
        ax.set_ylabel("y coordinate of projection [m]", fontsize=12)
        ax.set_title(f"ID: {feature_id}, {glacier_name}", fontsize=16)

        plt.tight_layout()
        output_path = os.path.join(plot_points_path, f"fl_{feature_id}_points_v3.pdf")
        plt.savefig(output_path, format="pdf", bbox_inches="tight")
        plt.close(fig)


        # === SAVE PLOT WITH LINES ===

        # Create output folder if not exists
        plot_lines_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Graphs/with Lines'
        os.makedirs(plot_lines_path, exist_ok=True)

        # Compute square bounds
        minx, miny, maxx, maxy = result.total_bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2

        width = maxx - minx
        height = maxy - miny
        max_extent = max(width, height) / 2 + 5000  # buffer of 5 km on each side

        minx = center_x - max_extent
        maxx = center_x + max_extent
        miny = center_y - max_extent
        maxy = center_y + max_extent

        # Flip y and Vv if needed (raster is top-down)
        if y[0] > y[-1]:
            y = y[::-1]
            Vv = Vv[::-1, :]

        # Wrap into xarray DataArray
        Vv = xr.DataArray(Vv, coords=[("y", y), ("x", x)])

        Vv_crop = Vv.sel(x=slice(minx, maxx), y=slice(miny, maxy))

        # Compute ratio for correct aspect
        width = maxx - minx
        height = maxy - miny
        # Check for valid numbers
        if width > 0 and height > 0:
            ratio = height / width
        else:
            raise ValueError("Invalid bounds: cannot compute aspect ratio.")

        fig, ax = plt.subplots(1, figsize=(12, 12))
        ax.set_aspect('equal', adjustable='box')

        # Background raster
        Vv_crop.plot(
            ax=ax,
            cmap="speed_colorblind",
            vmin=10,
            vmax=1500,
            zorder=1,
            cbar_kwargs={
                "shrink": 0.6,
                "label": r"velocity [m a$^{-1}$]",
                "pad": 0.02
            }
        )

        # Contours
        contour.plot(ax=ax, color="b", edgecolor="k", linewidth=0.25, zorder=2)
        contour100m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)
        contour1500m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)

        # NWCW outline (on top of contours)
        NWCW.plot(ax=ax, facecolor="none", edgecolor="g", linewidth=1, zorder=3)

        # Flowlines
        result_lines.plot(ax=ax, color="w", edgecolor="k", linewidth=0.75, zorder=4)

        # Starting points (on top)
        starting_points_df.plot(ax=ax, color="black", edgecolor="k", linewidth=0.01, markersize=0.5, zorder=5)

        # Labels
        ax.set_xlabel("x coordinate of projection [m]", fontsize=12)
        ax.set_ylabel("y coordinate of projection [m]", fontsize=12)
        ax.set_title(f"ID: {feature_id}, {glacier_name}", fontsize=16)

        plt.tight_layout()
        output_path = os.path.join(plot_lines_path, f"fl_{feature_id}_lines_v3.pdf")
        plt.savefig(output_path, format="pdf", bbox_inches="tight")
        plt.close(fig)

        # === SAVE PLOT WITH LINES (WITH CHAD'S IDs) ===

        # Create output folder if not exists
        plot_lines_chad_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Graphs/with Lines Chad'
        os.makedirs(plot_lines_chad_path, exist_ok=True)

        # Compute square bounds
        minx, miny, maxx, maxy = result.total_bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2

        width = maxx - minx
        height = maxy - miny
        max_extent = max(width, height) / 2 + 5000  # buffer of 5 km on each side

        minx = center_x - max_extent
        maxx = center_x + max_extent
        miny = center_y - max_extent
        maxy = center_y + max_extent

        # Flip y and Vv if needed (raster is top-down)
        if y[0] > y[-1]:
            y = y[::-1]
            Vv = Vv[::-1, :]

        # Wrap into xarray DataArray
        Vv = xr.DataArray(Vv, coords=[("y", y), ("x", x)])

        Vv_crop = Vv.sel(x=slice(minx, maxx), y=slice(miny, maxy))

        # Compute ratio for correct aspect
        width = maxx - minx
        height = maxy - miny
        # Check for valid numbers
        if width > 0 and height > 0:
            ratio = height / width
        else:
            raise ValueError("Invalid bounds: cannot compute aspect ratio.")

        fig, ax = plt.subplots(1, figsize=(12, 12))
        ax.set_aspect('equal', adjustable='box')

        # Background raster
        Vv_crop.plot(
            ax=ax,
            cmap="speed_colorblind",
            vmin=10,
            vmax=1500,
            zorder=1,
            cbar_kwargs={
                "shrink": 0.6,
                "label": r"velocity [m a$^{-1}$]",
                "pad": 0.02
            }
        )

        # Contours
        contour.plot(ax=ax, color="b", edgecolor="k", linewidth=0.25, zorder=2)
        contour100m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)
        contour1500m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)

        # NWCW outline (on top of contours)
        NWCW.plot(ax=ax, facecolor="none", edgecolor="g", linewidth=1, zorder=3)

        # Flowlines
        result_lines.plot(ax=ax, color="w", edgecolor="k", linewidth=0.75, zorder=4)

        # Starting points (on top)
        starting_points_df.plot(ax=ax, color="black", edgecolor="k", linewidth=0.01, markersize=0.5, zorder=5)

        # Labels
        ax.set_xlabel("x coordinate of projection [m]", fontsize=12)
        ax.set_ylabel("y coordinate of projection [m]", fontsize=12)
        ax.set_title(f"ID: {feature_id + 1}, {glacier_name}", fontsize=16)

        plt.tight_layout()
        output_path = os.path.join(plot_lines_chad_path, f"fl_{feature_id}_lines_chad_v3.pdf")
        plt.savefig(output_path, format="pdf", bbox_inches="tight")
        plt.close(fig)

        print(f"For Glacier {glacier_name} (ID:{feature_id}) saved plots).") 

        # === COMPUTE AND SAVE GLACIER STATISTICS ===

        # Compute per-glacier statistics
        stats_row = {
            "glacier_ID": glacier_id,
            "feature_ID": feature_id,
            "glacier_name": glacier_name,
            "num_sp": len(starting_points_df),
            "num_fl": len(result_lines),
            "num_fl_points": len(result),
            "failed_flowlines": len(starting_points_df) - len(result_lines)
        }

        if not flowline_stats.empty:
            stats_row["fastest_fl_sum"] = flowline_stats.sort_values("v_sum", ascending=False).iloc[0]["pathline_id"]
            stats_row["fastest_fl_avg"] = flowline_stats.sort_values("v_avg", ascending=False).iloc[0]["pathline_id"]
            stats_row["longest_fl"] = flowline_stats.sort_values("stopped_at_distance", ascending=False).iloc[0]["pathline_id"]
        else:
            stats_row["fastest_fl_sum"] = None
            stats_row["fastest_fl_avg"] = None
            stats_row["longest_fl"] = None

        # Append to global stats list
        if "all_glacier_stats" not in globals():
            all_glacier_stats = []

        all_glacier_stats.append(stats_row) 


    except Exception as e:
        print(print(f"Failed processing Glacier {glacier_name} (ID: {feature_id}): {e}"))

stats_df = pd.DataFrame(all_glacier_stats)
stats_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Statistics/Glaciers'
os.makedirs(stats_path, exist_ok=True)
filepath = os.path.join(stats_path, "stats_gl_1_v3.csv")
stats_df.to_csv(filepath, index=False)

print(f"Saved all-glacier statistics.")        


Picked Flowlines Extraction

In [ ]:
import os
import pandas as pd
import geopandas as gpd

# Input paths
picks_csv_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/fl_picks_3.1.csv'
flowlines_dir = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as Lines'
output_dir = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Picks'
os.makedirs(output_dir, exist_ok=True)

# Load picks CSV
picks_df = pd.read_csv(picks_csv_path)

# Iterate over each row (each glacier/feature_ID)
for _, row in picks_df.iterrows():
    feature_id = row['feature_ID']
    flowline_file = os.path.join(flowlines_dir, f"fl_{feature_id}_lines_v3.geojson")

    if not os.path.exists(flowline_file):
        print(f"⚠️ Flowline file missing: {flowline_file}")
        continue

    # Load flowlines for current feature_ID
    gdf = gpd.read_file(flowline_file)

    # Check all pathline_id columns (pathline_id_1, pathline_id_2, ...)
    for col in row.index:
        if col.startswith("pathline_id_") and pd.notna(row[col]):
            pathline_id = int(row[col])
            flowline_number = col.split("_")[-1]  # e.g., '1', '2', etc.

            # Filter flowline with specific pathline_id
            subset = gdf[gdf['pathline_id'] == pathline_id]
            if subset.empty:
                print(f"Pathline_id {pathline_id} not found in fl_{feature_id}_lines_v3.geojson")
                continue

            # Save each selected flowline as separate GeoJSON
            out_name = f"gl_{feature_id}_{flowline_number}.geojson"
            out_path = os.path.join(output_dir, out_name)
            subset.to_file(out_path, driver='GeoJSON')
            print(f"Saved: {out_name}")

Flowline Graphs with Picked Flowlines

In [ ]:
import os
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import rasterio
import numpy as np

# === Paths ===
all_fl_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as Lines/Store'
picked_fl_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Picks'
sp_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Input/Starting Points'
out_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Graphs with Picks/with Lines'
out_chad_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Graphs with Picks/with Lines Chad'
titles_csv = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/fl_picks_3.1.csv'

os.makedirs(out_path, exist_ok=True)
os.makedirs(out_chad_path, exist_ok=True)

# === Static overlays ===
NWCW = gp.read_file('data/kuba-flowlines/NW-CW_Basins.shp')
contour100m = gp.read_file('data/kuba-flowlines/contour_100masl_arcticdem_10m_v4.1_nwcw.shp')
contour1500m = gp.read_file('data/kuba-flowlines/contour_1500masl_arcticdem_10m_v4.1_nwcw.shp')
contour = gp.read_file('data/kuba-flowlines/contour_every100_arcticdem_10m_v4.1_nwcw.shp')

# === Raster ===
with rasterio.open("data/kuba-flowlines/GL_vel_mosaic_Annual_01Dec15_30Nov21_vv_v05.0_nwcw_average.tif") as src_vv:
    Vv = src_vv.read(1)
    transform = src_vv.transform
    x = (np.arange(src_vv.width) * transform.a) + transform.c
    y = (np.arange(src_vv.height) * transform.e) + transform.f

if y[0] > y[-1]:
    y = y[::-1]
    Vv = Vv[::-1, :]

Vv = xr.DataArray(Vv, coords=[("y", y), ("x", x)])

# === Metadata ===
titles_df = pd.read_csv(titles_csv)

# === Loop ===
for _, row in titles_df.iterrows():
    feature_id = int(row["feature_ID"])
    glacier_name = row["glacier_name"]

    try:
        fl_file = f"fl_{feature_id}_lines_v3.geojson"
        all_fl = gp.read_file(os.path.join(all_fl_path, fl_file))

        sp_file = f"sp_{feature_id}.geojson"
        starting_points_df = gp.read_file(os.path.join(sp_path, sp_file))

        picks = []
        for fname in os.listdir(picked_fl_path):
            if fname.startswith(f"gl_{feature_id}_") and fname.endswith(".geojson"):
                picks.append(gp.read_file(os.path.join(picked_fl_path, fname)))

        if not picks:
            print(f"No picks found for glacier {feature_id}. Skipping plot.")
            continue

        picks_gdf = pd.concat(picks).reset_index(drop=True)

        # Bounds
        minx, miny, maxx, maxy = all_fl.total_bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        width = maxx - minx
        height = maxy - miny
        max_extent = max(width, height) / 2 + 5000
        minx = center_x - max_extent
        maxx = center_x + max_extent
        miny = center_y - max_extent
        maxy = center_y + max_extent

        Vv_crop = Vv.sel(x=slice(minx, maxx), y=slice(miny, maxy))

        for suffix, title_id, out_dir in [("", feature_id, out_path), ("_chad", feature_id + 1, out_chad_path)]:
            fig, ax = plt.subplots(1, figsize=(12, 12))
            ax.set_aspect('equal', adjustable='box')

            Vv_crop.plot(
                ax=ax,
                cmap="speed_colorblind",
                vmin=10,
                vmax=1500,
                zorder=1,
                cbar_kwargs={"shrink": 0.6, "label": r"velocity [m a$^{-1}$]", "pad": 0.02}
            )

            contour.plot(ax=ax, color="b", edgecolor="k", linewidth=0.25, zorder=2)
            contour100m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)
            contour1500m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=2)
            NWCW.plot(ax=ax, facecolor="none", edgecolor="g", linewidth=1, zorder=3)
            all_fl.plot(ax=ax, color="w", edgecolor="k", linewidth=0.75, zorder=4)
            picks_gdf.plot(ax=ax, color="black", edgecolor="k", linewidth=2.5, zorder=5)
            starting_points_df.plot(ax=ax, color="black", edgecolor="k", linewidth=0.01, markersize=0.5, zorder=6)

            ax.set_xlabel("x coordinate of projection [m]", fontsize=12)
            ax.set_ylabel("y coordinate of projection [m]", fontsize=12)
            ax.set_title(f"ID: {title_id}, {glacier_name}", fontsize=16)

            plt.tight_layout()
            filename = f"fl_{feature_id}_picked_lines{suffix}_v3.pdf"
            output_path = os.path.join(out_dir, filename)
            plt.savefig(output_path, format="pdf", bbox_inches="tight")
            plt.close(fig)

        print(f"Saved plots for Glacier {glacier_name} ({feature_id}).")

    except Exception as e:
        print(f"Error processing Glacier {glacier_name} ({feature_id}): {e}")

Flowline Graphs with Picked Flowlines and Elevation Buffers

In [ ]:
import os
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import rasterio

# === Paths ===
all_fl_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Output/Flowlines/as Lines'
picked_fl_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Picks/Store'
int_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Points/Intersections Cleaned'
buff_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Points/Buffers'
out_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Graphs with Picks and Buffers/with Lines'
out_chad_path = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/Graphs with Picks and Buffers/with Lines Chad'
titles_csv = '/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Flowlines of Interest/fl_picks_3.1.csv'

os.makedirs(out_path, exist_ok=True)
os.makedirs(out_chad_path, exist_ok=True)

# === Static overlays ===
NWCW = gp.read_file('data/kuba-flowlines/NW-CW_Basins.shp')
contour100m = gp.read_file('data/kuba-flowlines/contour_100masl_arcticdem_10m_v4.1_nwcw.shp')
contour1500m = gp.read_file('data/kuba-flowlines/contour_1500masl_arcticdem_10m_v4.1_nwcw.shp')
contour = gp.read_file('data/kuba-flowlines/contour_every100_arcticdem_10m_v4.1_nwcw.shp')

# === Raster ===
with rasterio.open("data/kuba-flowlines/GL_vel_mosaic_Annual_01Dec15_30Nov21_vv_v05.0_nwcw_average.tif") as src_vv:
    Vv = src_vv.read(1)
    transform = src_vv.transform
    x = (np.arange(src_vv.width) * transform.a) + transform.c
    y = (np.arange(src_vv.height) * transform.e) + transform.f

if y[0] > y[-1]:
    y = y[::-1]
    Vv = Vv[::-1, :]

Vv = xr.DataArray(Vv, coords=[("y", y), ("x", x)])

# === Metadata ===
titles_df = pd.read_csv(titles_csv)

# === Loop ===
for _, row in titles_df.iterrows():
    feature_id = int(row["feature_ID"])
    glacier_name = row["glacier_name"]

    try:
        picks = []
        intersections = []
        buffers = []

        for fname in os.listdir(picked_fl_path):
            if fname.startswith(f"gl_{feature_id}_") and fname.endswith(".geojson"):
                fl_id = fname.split("_")[-1].split(".")[0]  # Extract flowline ID
                picks.append(gp.read_file(os.path.join(picked_fl_path, fname)))

                int_path_file = f"intsect_clean_gl_{feature_id}_{fl_id}.geojson"
                buff_path_file = f"buff_intsect_clean_gl_{feature_id}_{fl_id}.geojson"

                if os.path.exists(os.path.join(int_path, int_path_file)):
                    intersections.append(gp.read_file(os.path.join(int_path, int_path_file)))
                if os.path.exists(os.path.join(buff_path, buff_path_file)):
                    buffers.append(gp.read_file(os.path.join(buff_path, buff_path_file)))

        if not picks:
            print(f"No picks found for glacier {feature_id}. Skipping plot.")
            continue

        picks_gdf = pd.concat(picks).reset_index(drop=True)
        intersections_gdf = pd.concat(intersections).reset_index(drop=True) if intersections else None
        buffers_gdf = pd.concat(buffers).reset_index(drop=True) if buffers else None

        minx, miny, maxx, maxy = picks_gdf.total_bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        width = maxx - minx
        height = maxy - miny
        max_extent = max(width, height) / 2 + 5000
        minx = center_x - max_extent
        maxx = center_x + max_extent
        miny = center_y - max_extent
        maxy = center_y + max_extent

        Vv_crop = Vv.sel(x=slice(minx, maxx), y=slice(miny, maxy))

        for suffix, title_id, out_dir in [("", feature_id, out_path), ("_chad", feature_id + 1, out_chad_path)]:
            fig, ax = plt.subplots(1, figsize=(12, 12))
            ax.set_aspect('equal', adjustable='box')

            Vv_crop.plot(
                ax=ax,
                cmap="speed_colorblind",
                vmin=10,
                vmax=1500,
                zorder=1,
                cbar_kwargs={"shrink": 0.6, "label": r"velocity [m a$^{-1}$]", "pad": 0.02}
            )

            contour.plot(ax=ax, color="b", edgecolor="k", linewidth=0.25, zorder=3)
            contour100m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=3)
            contour1500m.plot(ax=ax, color="b", edgecolor="k", linewidth=1, zorder=3)
            NWCW.plot(ax=ax, facecolor="none", edgecolor="g", linewidth=1, zorder=4)
            picks_gdf.plot(ax=ax, color="black", edgecolor="k", linewidth=2.5, zorder=5)

            if buffers_gdf is not None:
                buffers_gdf.plot(ax=ax, color="green", edgecolor="k", linewidth=0.5, alpha=0.5, zorder=2)

            if intersections_gdf is not None:
                intersections_gdf.plot(ax=ax, color="white", edgecolor="k", linewidth=0.01, markersize=5, zorder=6)

            ax.set_xlabel("x coordinate of projection [m]", fontsize=12)
            ax.set_ylabel("y coordinate of projection [m]", fontsize=12)
            ax.set_title(f"ID: {title_id}, {glacier_name}", fontsize=16)

            plt.tight_layout()
            filename = f"fl_{feature_id}_buffers{suffix}_v3.pdf"
            output_path = os.path.join(out_dir, filename)
            plt.savefig(output_path, format="pdf", bbox_inches="tight")
            plt.close(fig)

        print(f"Saved plots for Glacier {glacier_name} ({feature_id}).")

    except Exception as e:
        print(f"Error processing Glacier {glacier_name} ({feature_id}): {e}")

Save Sampling Points

In [ ]:
import os
import geopandas as gp
import pandas as pd
from pathlib import Path

# === Paths ===
input_dir = Path('/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Points/Random Points')
geojson_out_dir = Path('/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Points/Sampling Points/as GeoJSON')
csv_out_dir = Path('/Users/jagon/Documents/Projects/Collabs/Jessica Badgeley/New Points v3/Points/Sampling Points/as CSV')

# Process each GeoJSON file
for file in input_dir.glob("rp_buff_intsect_clean_gl_*.geojson"):
    try:
        gdf = gp.read_file(file)

        # Extract glacier and flowline ID from filename
        parts = file.stem.split("gl_")[-1].split("_")
        glacier_id = parts[0]
        flowline_id = parts[1]
        base_name = f"gl_{glacier_id}_{flowline_id}"

        # Create subdirectories for glacier ID and flowline ID
        glacier_geojson_dir = geojson_out_dir / f"Glacier {glacier_id} Flowline {flowline_id}"
        glacier_csv_dir = csv_out_dir / f"Glacier {glacier_id} Flowline {flowline_id}"
        glacier_geojson_dir.mkdir(parents=True, exist_ok=True)
        glacier_csv_dir.mkdir(parents=True, exist_ok=True)

        # Group by elevation
        grouped = gdf.groupby(gdf["ELEV"].astype(int))

        for elev, group in grouped:
            group = group.copy()
            elev_str = str(int(elev))
            geojson_path = glacier_geojson_dir / f"{base_name}_{elev_str}.geojson"
            csv_path = glacier_csv_dir / f"{base_name}_{elev_str}.csv"

            # Save grouped GeoDataFrame to GeoJSON
            group.to_file(geojson_path, driver="GeoJSON")

            # Add x, y in EPSG:3413
            group["x_3413"] = group.geometry.x
            group["y_3413"] = group.geometry.y

            # Reproject and extract x, y in EPSG:4326
            group_wgs84 = group.to_crs(epsg=4326)
            group["x_4326"] = group_wgs84.geometry.x
            group["y_4326"] = group_wgs84.geometry.y

            # Save to CSV with coordinates
            group.drop(columns="geometry").to_csv(csv_path, index=False)

        print(f"Processed {file.name} into {len(grouped)} elevation groups.")

    except Exception as e:
        print(f"Error processing {file.name}: {e}")

